# Lab orders

In [1]:
%matplotlib inline
# notebook
import matplotlib.pylab as pylab
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.cluster import KMeans
from collections import Counter
import numpy as np
from sklearn.decomposition import PCA
from datetime import datetime, timedelta


#make the graphs bigger
pylab.rcParams['figure.figsize'] = (32.0, 24.0)
pylab.rcParams['font.size'] = 24

In [3]:
cohort = []
ETOH = []
hw = []
labOrders = []
medOrders = []
prescriptions = []
radiology = []
tobacco = []
vitals = []

def fileToList(fname):
    dataArray = []
    with open(fname) as f:
        for line in f:
            new = line.strip()
            new = new[1:len(new) - 1]
            new = new.split('","')
            if (len(new) > 1):
                dataArray.append(new)
    return dataArray

def fileToDict(fname):
    dataDict = dict()
    with open(fname) as f:
        for line in f:
            new = line.strip()
            new = new[1:len(new) - 1]
            new = new.split('","')
            if (len(new) > 1):
                if not(new[0] in dataDict):
                    dataDict[new[0]] = [new]
                else:
                    dataDict[new[0]].append(new)
    return dataDict

def largeFileToDict(fname, upto):
    dataDict = dict()
    with open(fname) as f:
        count = 0
        for line in f:
            new = line.strip()
            new = new[1:len(new) - 1]
            new = new.split('","')
            if (len(new) > 1):
                if not(new[0] in dataDict):
                    dataDict[new[0]] = [new]
                else:
                    dataDict[new[0]].append(new)
            count = count + 1
            if (count == upto):
                break;
    return dataDict

def largeFileToList(fname, upto):
    dataArray = []
    with open(fname) as f:
        count = 0
        for line in f:
            new = line.strip()
            new = new[1:len(new) - 1]
            new = new.split('","')
            if (len(new) > 1):
                dataArray.append(new)
            count = count + 1
            if (count == upto):
                break;
    return dataArray

In [4]:
fname = "2018-4169_Med_Orders.txt"
medOrdersList = largeFileToList(fname, 100)

In [5]:
print("Hi")

Hi


In [6]:
fname = "2018-4169_Med_Orders.txt"
medOrders = fileToDict(fname)

fname = "2018-4169_Prescriptions.txt"
prescriptions = fileToList(fname)

In [7]:
fname = "2018-4169_Prescriptions.txt"
preScriptions = fileToDict(fname)

In [70]:
fname = "2018-4169_Vital_Signs.txt"
vitals = largeFileToDict(fname, 100000)

In [9]:
fname = "2018-4169_Cohort.txt"
cohort = fileToList(fname)

In [10]:
print(len(vitals.keys()))

100


In [11]:
fname = "2018-4169_Lab_Orders_Performed.txt"
labOrdersList = fileToList(fname)

fname = "2018-4169_Lab_Orders_Performed.txt"
labOrders = fileToDict(fname)

In [12]:
print(list(labOrders.keys())[0:10])

['person_de_id', '0.52938512525816972', '0.69021487534053338', '0.17082648770834272', '0.42715738434469491', '0.36753633968201543', '0.2520880241670635', '0.36152515055660644', '0.1594862688030885', '0.036014403266842']


In [13]:
print(labOrdersList[2])

['0.69021487534053338', '0.91501930382036079', 'Comprehensive Metabolic Panel', 'STAT', '2017-10-22 07:47:00']


In [8]:
orders = [item[2] if len(item) > 2 else item[1] for item in labOrdersList[1:]]
#ordered = Counter(labOrders[1:])
#print(ordered)

In [9]:
ordered = Counter(orders)
print(ordered)

Counter({'CBC With Diff': 197257, 'Comprehensive Metabolic Panel': 145427, 'Basic Metabolic Panel': 81047, 'Lactic Acid': 75311, 'Glucose Point of Care Testing (LIO)': 57797, 'PT/INR/PTT': 52514, 'Urinalysis with Reflex to Culture, Random Urine': 47872, 'Urinalysis with Microscopic, Random Urine': 42644, 'Magnesium, Plasma': 41329, 'Hemoglobin, Hemocue Point of Care Testing (LIO)': 40108, 'Troponin I': 38984, 'Pregnancy, Urine (Qual)': 36963, 'Magnesium': 36842, 'Troponin I, Plasma': 36360, 'Phosphorus, Plasma': 35003, 'Drug Screen Rapid Panel 10 No Confirmation, Urine': 34221, 'Urinalysis with Reflex Culture': 34202, 'Glucose, Bedside (LIO)': 33818, 'Lipase, Plasma': 30447, 'Phosphorus': 30016, 'Lipase': 29978, 'Bacterial Culture Urine Quantitative': 28037, 'Type, Screen + Hold (Pretransfusion Testing)': 27181, 'GFR Calculation (LIO)': 26474, 'Bacterial Culture Blood #1': 21436, 'Bacterial Culture Blood #2': 19796, 'Beta hCG': 19110, 'CBC Without Diff': 18975, 'Glucose, Point of Care 

In [10]:
listed = sorted(list(ordered.items()), key=lambda x: x[1])[::-1]

In [11]:
print(listed[0:20])

[('CBC With Diff', 197257), ('Comprehensive Metabolic Panel', 145427), ('Basic Metabolic Panel', 81047), ('Lactic Acid', 75311), ('Glucose Point of Care Testing (LIO)', 57797), ('PT/INR/PTT', 52514), ('Urinalysis with Reflex to Culture, Random Urine', 47872), ('Urinalysis with Microscopic, Random Urine', 42644), ('Magnesium, Plasma', 41329), ('Hemoglobin, Hemocue Point of Care Testing (LIO)', 40108), ('Troponin I', 38984), ('Pregnancy, Urine (Qual)', 36963), ('Magnesium', 36842), ('Troponin I, Plasma', 36360), ('Phosphorus, Plasma', 35003), ('Drug Screen Rapid Panel 10 No Confirmation, Urine', 34221), ('Urinalysis with Reflex Culture', 34202), ('Glucose, Bedside (LIO)', 33818), ('Lipase, Plasma', 30447), ('Phosphorus', 30016)]


In [12]:
print(listed[0:30])

[('CBC With Diff', 197257), ('Comprehensive Metabolic Panel', 145427), ('Basic Metabolic Panel', 81047), ('Lactic Acid', 75311), ('Glucose Point of Care Testing (LIO)', 57797), ('PT/INR/PTT', 52514), ('Urinalysis with Reflex to Culture, Random Urine', 47872), ('Urinalysis with Microscopic, Random Urine', 42644), ('Magnesium, Plasma', 41329), ('Hemoglobin, Hemocue Point of Care Testing (LIO)', 40108), ('Troponin I', 38984), ('Pregnancy, Urine (Qual)', 36963), ('Magnesium', 36842), ('Troponin I, Plasma', 36360), ('Phosphorus, Plasma', 35003), ('Drug Screen Rapid Panel 10 No Confirmation, Urine', 34221), ('Urinalysis with Reflex Culture', 34202), ('Glucose, Bedside (LIO)', 33818), ('Lipase, Plasma', 30447), ('Phosphorus', 30016), ('Lipase', 29978), ('Bacterial Culture Urine Quantitative', 28037), ('Type, Screen + Hold (Pretransfusion Testing)', 27181), ('GFR Calculation (LIO)', 26474), ('Bacterial Culture Blood #1', 21436), ('Bacterial Culture Blood #2', 19796), ('Beta hCG', 19110), ('CBC

In [14]:
def difference(l1,l2):
    return 1 - len([b for a,b in zip(l1,l2) if b != a]) 

def runModels(Xtr, patientBase, printOn=True):
    kf = KFold(n_splits=5, shuffle=True)
    Ytr = patientBase
    differencesLR = []
    differencesKM = []

    for train_index, test_index in kf.split(Xtr):
        X_train, X_test = [Xtr[i] for i in train_index], [Xtr[i] for i in test_index]
        y_train, y_test = [Ytr[i] for i in train_index], [Ytr[i] for i in test_index]
    
        if (1 in y_train and 0 in y_train):
            lr = LogisticRegression()
            lr.fit(X_train, y_train)
            #PredLR = lr.predict(X_test)
            differencesLR.append(lr.score(X_test, y_test))
            #differencesLR.append(difference(PredLR, y_test)/len(y_test))
    
            #km = KMeans(n_clusters=2)
            #km.fit(X_train, y_train)
            #PredKM = km.predict(X_test)
            #differencesKM.append(difference(PredKM, y_test)/len(y_test))
            
            neigh = KNeighborsClassifier(n_neighbors=5)
            neigh.fit(X_train, y_train)
            differencesKM.append(neigh.score(X_test,y_test))
    if(printOn):
        print(differencesLR)
        print(differencesKM)
    
    return (np.mean(differencesLR) + np.mean(differencesKM))/2

In [15]:
def abstractFreqFlyers(organised, gap, visits):
    frequentFlyers = []
    for p in organised.keys():
        dates = [datetime.strptime(item[8][0:10], '%Y-%m-%d') for item in organised[p]]
        try:
            for i in range(len(dates)):
                modified_date = dates[i] + timedelta(days=gap)
                if dates[i+visits-1] <= modified_date:
                    frequentFlyers.append(p)
                    break
        except:
            pass
    return frequentFlyers

def retrieveRegulars(org, frequents):
    regularsBase = []
    newOrder = Counter(frequents)
    for k in org.keys():
        if not(k in newOrder):
            regularsBase.append(k)
    
    return list(np.random.choice(regularsBase, len(frequents)))

def organiseCohortData(myCohort):
    organised = dict()
    for admission in myCohort:
        if admission[0] in organised:
            organised[admission[0]].append(admission[1:])
        else:
            organised[admission[0]] = [admission[1:]]
            
    
    for p in organised.keys():
        organised[p] = sorted(organised[p], key = lambda x: datetime.strptime(x[8][0:10], '%Y-%m-%d'))
    return organised

In [16]:
def safeIndex(myList, item):
    if item in myList:
        return myList.index(item)
    else:
        return -1

def vectorise(uid, n, dataSet, dataIndex, crushedListed):
    vector = [0] * n
    if (uid in dataSet):
        data = [item[dataIndex] if len(item) > dataIndex else item[dataIndex-1] for item in dataSet[uid]]
        data = list(map(lambda x: safeIndex(crushedListed, x), data))
        for i in data:
            if not (i == -1):
                vector[i] = 1;
    return vector
        
        

def runAnalysis(freqs, regs, n, dataSet, dataIndex, countedUp):
    
    listed = sorted(list(countedUp.items()), key=lambda x: x[1])[::-1]
    listed = listed[0:n]
    crushedListed = [item[0] for item in listed]
        
    frequentData = list(map(lambda x: vectorise(x, n, dataSet, dataIndex, crushedListed), freqs))
    regsData = list(map(lambda x: vectorise(x, n, dataSet, dataIndex, crushedListed), regs))
    
    Y = ([1] * len(freqs)) + ([0] * len(regs))
    X = frequentData + regsData
        
    print(len(Y))
    print(len(X))
    
    print("Printing for all 20 features")
    runModels(X,Y)
    print("5 most influencial features")
    indie = []
    for i in range(20):
        indie.append((runModels([[item[i]] for item in X], Y, printOn=False), i))
    indie = sorted(indie, key=lambda x: x[0])[::-1]
    indie = indie[0:5]
    for col in indie:
        print(str(col[0]) + "  " + str(col[1]))
        


In [17]:
# N=
organised = organiseCohortData(cohort[1:])
freqs = abstractFreqFlyers(organised, 365, 6)
regs = retrieveRegulars(organised, freqs)


In [18]:
allOrders = [item[2] if len(item) > 2 else item[1] for item in labOrdersList[1:]]
countedUp = Counter(allOrders)

runAnalysis(freqs, regs, 30, labOrders, 2, countedUp)

6362
6362
Printing for all 20 features
[0.87745483110761979, 0.86645718774548308, 0.88993710691823902, 0.87342767295597479, 0.87342767295597479]
[0.84917517674783971, 0.84446190102120977, 0.85534591194968557, 0.84040880503144655, 0.85220125786163525]
5 most influencial features
0.737505249324  1
0.70323926544  0
0.648775116473  2
0.61435066475  3
0.607433290351  18


In [19]:
allOrders = [item[3] if len(item) > 2 else item[1] for item in medOrdersList[1:]]
countedUp = Counter(allOrders)

runAnalysis(freqs, regs, 30, medOrders, 3, countedUp)

6362
6362
Printing for all 20 features
[0.84996072270227807, 0.85153181461115479, 0.85691823899371067, 0.8529874213836478, 0.84433962264150941]
[0.812254516889238, 0.80911233307148467, 0.8058176100628931, 0.81367924528301883, 0.79088050314465408]
5 most influencial features
0.778531745444  1
0.637302625403  4
0.624486801346  3
0.62236693889  0
0.605466399383  6


In [21]:
allOrders = [item[3] if len(item) > 2 else item[1] for item in prescriptions[1:]]
countedUp = Counter(allOrders)

runAnalysis(freqs, regs, 40, preScriptions, 3, countedUp)

6362
6362
Printing for all 20 features
[0.83110761979575809, 0.82089552238805974, 0.839622641509434, 0.83333333333333337, 0.81525157232704404]
[0.53260015710919084, 0.75569520816967795, 0.78066037735849059, 0.76100628930817615, 0.76572327044025157]
5 most influencial features
0.614271492587  0
0.575289392165  3
0.553441271794  4
0.546682797532  10
0.54550429333  13


In [22]:
print(len(vitals.keys()))
#print(vitals[list(vitals.keys())[0]][0])
#print(vitals[list(vitals.keys())[20]])

100


In [72]:
#xs = [(item[3], item[4]) for item in ]
def vectoriseModded(uid, n, dataSet, crushedListed):
    vector = [0] * n
    if (uid in dataSet):
        categories = [item[3] if len(item) > 3 else item[2] for item in dataSet[uid]]
        tuples = [(item[3], item[4]) if len(item) > 2 else item[2] for item in dataSet[uid]]
        vector_data = list(map(lambda x: safeIndex(crushedListed, x), categories))
        for k, i in enumerate(categories):
            if not(vector_data[k] == -1):
                values = [x[1] if (x[0] == i) else 0 for x in tuples]
                try:
                    temp = list(map(float, values))
                    vector[vector_data[k]] = sum(temp) / len(list(filter(lambda x: x != 0, values)))
                except:
                    vector[vector_data[k]] = 10
    return vector

In [75]:
def runVitalsAnalysis(freqs, regs, vitalsDict, n, crushedListed):
    frequentData = list(map(lambda x: vectoriseModded(x, n, vitalsDict, crushedListed), freqs))
    regsData = list(map(lambda x: vectoriseModded(x, n, vitalsDict, crushedListed), regs))
    
    Y = ([1] * len(freqs)) + ([0] * len(regs))
    X = frequentData + regsData
    
    
    print(frequentData[243])
    print(len(Y))
    print(len(X))
    
    print("Printing for all 20 features")
    runModels(X,Y)
    print("5 most influencial features")
    indie = []
    for i in range(20):
        indie.append((runModels([[item[i]] for item in X], Y, printOn=False), i))
    indie = sorted(indie, key=lambda x: x[0])[::-1]
    indie = indie[0:5]
    for col in indie:
        print(str(col[0]) + "  " + str(col[1]))
        
totalList = []
for i in vitals.keys():
    totalList = totalList + [item[3] if len(item) > 2 else item[1] for item in vitals[i]]

countedUp = Counter(totalList)
n = 30

listed = sorted(list(countedUp.items()), key=lambda x: x[1])[::-1]
listed = listed[0:n]
crushedListed = [item[0] for item in listed]
    
runVitalsAnalysis(freqs, regs, vitals, 30, crushedListed)

[0, 102.0, 0, 0, 0, 97.3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6362
6362
Printing for all 20 features
[0.83817753338570311, 0.85938727415553806, 0.86399371069182385, 0.85613207547169812, 0.83883647798742134]
[0.80125687352710129, 0.54595443833464252, 0.50157232704402521, 0.82154088050314467, 0.81132075471698117]
5 most influencial features
0.597218290375  2
0.592421395999  0
0.592266139511  4
0.590931390713  1
0.589829526647  3
